In [8]:
import os
import pandas as pd
import arcpy
import arcpy.mp as mp
import re
import requests
import numpy as np
import geopandas
from arcpy.sa import *
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

#### Define function to reproject raster data and store within a mobile geodatabase

In [9]:

def func_import_raster(raster_path, mgdb_path, reference_fc):
    """
    Copies a raster to a mobile geodatabase and reprojects it to match the spatial reference
    of a reference feature class. Saves the reprojected raster with '_reproj' appended to the name.

    Parameters:
        raster_path (str): Path to the original .tif raster.
        mgdb_path (str): Path to the mobile geodatabase (.geodatabase).
        reference_fc (str): Full path to the feature class in the same geodatabase to match projection.
    
    Returns:
        Tuple[str, str]: Paths to the original raster in the GDB and the reprojected raster in the GDB.
    """
    
    # Extract base name (no extension)
    raster_name = os.path.splitext(os.path.basename(raster_path))[0]
    raster_in_gdb = os.path.join(mgdb_path, raster_name)

    # Copy raster to GDB if it doesn't exist
    if not arcpy.Exists(raster_in_gdb):
        arcpy.management.CopyRaster(raster_path, raster_in_gdb)
    
    # Get spatial reference of the reference feature class
    ref_sr = arcpy.Describe(reference_fc).spatialReference

    # Reproject raster to match feature class
    raster_reproj_name = f"{raster_name}_reproj"
    raster_reproj_gdb = os.path.join(mgdb_path, raster_reproj_name)

    arcpy.management.ProjectRaster(
        in_raster=raster_in_gdb,
        out_raster=raster_reproj_gdb,
        out_coor_system=ref_sr,
        resampling_type="BILINEAR"
    )

    return raster_in_gdb, raster_reproj_gdb

#### Get the path of this notebook and use it to generate path for input shapefile:

In [10]:
current_dir = os.getcwd()
shapefile_cbg = r"input\cbg_kontur.shp"
shapefile_cbg_path = os.path.join(current_dir, shapefile_cbg)
output_folder_path = os.path.join(current_dir, "output")
print(f"Location of this notebook: {current_dir}")
print(f"Location of CBG SHP: {shapefile_cbg_path}")
print(f"Location for output: {output_folder_path}")

Location of this notebook: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis
Location of CBG SHP: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\cbg_kontur.shp
Location for output: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\output


#### Set up ArcPy Workspace:

In [11]:
arcpy.env.overwriteOutput = True
workspace = os.path.dirname(shapefile_cbg_path)
arcpy.env.workspace = workspace

#### Load shapefile into a layer and check spatial reference etc:

In [12]:
cbg_layer = shapefile_cbg_path
desc = arcpy.Describe(cbg_layer)
fields = [f.name for f in arcpy.ListFields(cbg_layer)]
count = arcpy.management.GetCount(cbg_layer)
print("Layer cbg_layer info:\n")
print("Shape type:", desc.shapeType)
print("Spatial reference:", desc.spatialReference.name)
print(f"Number of features: {count[0]}\n")
print(f"Field List: {fields}")


Layer cbg_layer info:

Shape type: Polygon
Spatial reference: WGS_1984_Web_Mercator_Auxiliary_Sphere
Number of features: 1057

Field List: ['FID', 'Shape', 'GEOID', 'GEO_ID', 'GEOIDFQ', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'NAMELSAD', 'P1_001N', 'trim_acres', 'pop_per_ac', 'pop_sqkm', 'Shape_Leng', 'Shape_Area', 'whole_bg_a', 'whole_bg_s', 'trim_bg_ac', 'trim_bg_sq', 'rainavgin']


#### Shapefile limits field names to only 10 char so we need to create a Mobile GDB as an intermediate step:

In [15]:
# Define mobile GDB path and name
mgdb_name = "cbg_analysis.geodatabase"
mgdb_path = os.path.join(workspace, mgdb_name)

# Create the mobile GDB (only once)
if not arcpy.Exists(mgdb_path):
    arcpy.management.CreateMobileGDB(workspace, mgdb_name)

# Copy the shapefile into the mobile GDB
cbg_fc = os.path.join(mgdb_path, "cbg_layer")
if not arcpy.Exists(cbg_fc):
    arcpy.management.CopyFeatures(shapefile_cbg_path, cbg_fc)

fields = [f.name for f in arcpy.ListFields(cbg_fc)]
print(f"Field List: {fields}")

Field List: ['OBJECTID', 'GEOID', 'GEO_ID', 'GEOIDFQ', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'NAMELSAD', 'P1_001N', 'trim_acres', 'pop_per_ac', 'pop_sqkm', 'Shape_Leng', 'whole_bg_a', 'whole_bg_s', 'trim_bg_ac', 'trim_bg_sq', 'rainavgin', 'Shape', 'MeanIsohyetAnn_Hawaii_in_mean', 'st_area(Shape)', 'st_perimeter(Shape)']


#### Define location/name of raster for analysis and fieldname to put resulting value into:

In [16]:
raster_path = os.path.join(workspace, "MeanIsohyetAnn_Hawaii_in.tif")
stat_type = "MEAN"  # other options: "MAX", "MIN", "SUM", etc.

# Create field name using raster base name and stat type
raster_name = os.path.splitext(os.path.basename(raster_path))[0]
output_field = f"{raster_name}_{stat_type.lower()}"


In [20]:
print(cbg_fc)
arcpy.Exists(r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\cbg_analysis.geodatabase\cbg_layer")

C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\cbg_analysis.geodatabase\cbg_layer


True

In [19]:
func_import_raster(raster_path, mgdb_path, cbg_fc)
print("Raster imported")

ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
No spatial reference exists.
Failed to execute (CopyRaster).


#### Create a temporary table to store zonal stats and run the analysis:

In [7]:
# Run Zonal Statistics as Table
zone_field = "OBJECTID"
zonal_table = os.path.join("in_memory", f"{raster_name}_zonal")

ZonalStatisticsAsTable(
    in_zone_data=cbg_fc,
    zone_field=zone_field,
    in_value_raster=raster_path,
    out_table=zonal_table,
    statistics_type=stat_type,
    ignore_nodata="DATA"
)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 010568: Invalid extent. Please check for zero length or width, or failure to project extent to output spatial reference. 
Failed to execute (ZonalStatisticsAsTable).


#### Join the statistics back to the CBG feature class

In [ ]:
# Add a field for the result (now allowed to use long names)
arcpy.management.AddField(cbg_fc, output_field, "DOUBLE")

# Join the zonal stats table to the feature class
arcpy.management.JoinField(
    in_data=cbg_fc,
    in_field=zone_field,
    join_table=zonal_table,
    join_field=zone_field,
    fields=[stat_type]
)

# Copy the stat value into the named field
arcpy.management.CalculateField(
    in_table=cbg_fc,
    field=output_field,
    expression=f"!{stat_type}!",
    expression_type="PYTHON3"
)

# Remove the join
arcpy.management.RemoveJoin(cbg_fc)

In [ ]:


fields2 = [f.name for f in arcpy.ListFields(cbg_fc)]
print(f"New field List: {fields2}")